In [2]:
import capytaine as cpy
import autograd.numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import block_diag
import xarray as xr
from math import comb

import wecopttool as wot

In [28]:
# load in the empirical data
datadir = 'data/pioneer_empirical_data/'

def _create_DataArrays(impedance, exc_coeff, omega):
    freq_attr = {'long_name': 'Wave frequency', 'units': 'rad/s'}
    dir_attr = {'long_name': 'Wave direction', 'units': 'rad'}
    dof_attr = {'long_name': 'Degree of freedom'}
    dof_names = ["Pitch",]
    directions = np.atleast_1d(0.0)

    dims_exc = ('omega', 'wave_direction', 'influenced_dof')
    coords_exc = [
        (dims_exc[0], np.squeeze(omega), freq_attr),
        (dims_exc[1], directions, dir_attr),
        (dims_exc[2], dof_names, dof_attr),
    ]
    attrs_exc = {'units': 'N/m', 'long_name': 'Excitation Coefficient'}
    exc_coeff = np.expand_dims(np.squeeze(exc_coeff), axis = [1,2])
    exc_coeff = xr.DataArray(exc_coeff, dims=dims_exc, coords=coords_exc,
                            attrs=attrs_exc, name='excitation coefficient')

    dims_imp = ('omega', 'radiating_dof', 'influenced_dof')
    coords_imp = [
        (dims_imp[0], np.squeeze(omega), freq_attr),
        (dims_imp[1], dof_names, dof_attr),
        (dims_imp[2], dof_names, dof_attr),
    ]
    attrs_imp = {'units': 'Ns/m', 'long_name': 'Intrinsic Impedance'}

    Zi = xr.DataArray(
        np.expand_dims(np.squeeze(impedance), axis=[1,2]),
        dims=dims_imp, coords=coords_imp, attrs=attrs_imp,
        name='Intrinsic impedance'
    )
    return exc_coeff, Zi

omega_data = np.loadtxt(datadir + "omega")

exc_coeff_data = (np.loadtxt(datadir + "exc_real") + 1j*np.loadtxt(datadir + "exc_imag")).reshape([-1, 1])
Zi_data = (np.loadtxt(datadir + "Zemp_real") + 1j*np.loadtxt(datadir + "Zemp_imag")).reshape([-1, 1, 1])
Zi_stiffness = np.loadtxt(datadir + "Zemp_stiffness").reshape([1, 1])

exc_coeff_data, Zi_data = _create_DataArrays(Zi_data, exc_coeff_data, omega_data)

empirical_data = xr.Dataset(data_vars=dict(
            exc_coeff_data_real=(["omega", "wave_direction", "influenced_dof"], np.real(exc_coeff_data.data)),
            exc_coeff_data_imag=(["omega", "wave_direction", "influenced_dof"], np.imag(exc_coeff_data.data)),
            Zi_data_real=(["omega", "radiating_dof", "influenced_dof"], np.real(Zi_data.data)),
            Zi_data_imag=(["omega", "radiating_dof", "influenced_dof"], np.imag(Zi_data.data)),
            Zi_stiffness=(["influenced_dof","radiating_dof"],Zi_stiffness)
        ),
        coords=dict(
            omega=omega_data,
            wave_direction=np.atleast_1d(0.0),
            influenced_dof=["Pitch",],
            radiating_dof=["Pitch",],
        ),
        attrs={'long_name': 'Excitation and Impedance Data'}
    )

print(empirical_data)
empirical_data.to_netcdf("pioneer_empirical_data.nc")

<xarray.Dataset>
Dimensions:              (omega: 128, wave_direction: 1, influenced_dof: 1,
                          radiating_dof: 1)
Coordinates:
  * omega                (omega) float64 0.03142 0.06283 0.09425 ... 3.99 4.021
  * wave_direction       (wave_direction) float64 0.0
  * influenced_dof       (influenced_dof) <U5 'Pitch'
  * radiating_dof        (radiating_dof) <U5 'Pitch'
Data variables:
    exc_coeff_data_real  (omega, wave_direction, influenced_dof) float64 -1.5...
    exc_coeff_data_imag  (omega, wave_direction, influenced_dof) float64 3.51...
    Zi_data_real         (omega, radiating_dof, influenced_dof) float64 5.421...
    Zi_data_imag         (omega, radiating_dof, influenced_dof) float64 -2.09...
    Zi_stiffness         (influenced_dof, radiating_dof) float64 6.582e+04
Attributes:
    long_name:  Excitation and Impedance Data
